# A Data Collector Module for Storing Hand Coordinates from Mediapipe's Hands Solution

## Objective: 
Live data collection of person's hands gestures for further use for classification of the gesture.

## How it Works:
After setting up the dependicies and ensuring that opencv is setup with mediapipe, you will choose a name for your gesture and run the last block of code for opencv to detectyou performing the hand gesture for about 30-60 seconds with different angles.

## Install dependencies

In [ ]:
!pip install mediapipe opencv-python pandas scikit-learn

In [1]:
import mediapipe as mp 
import cv2

In [2]:
mp_drawing = mp.solutions.drawing_utils # Drawing helper
mp_hands = mp.solutions.hands

## Ensuring that opencv and mediapipe are working.
This is not for detection, only a test run for visualization. Please be sure that before pressing quit to terminate the run of the code block below, your hand was being detected and `results` was not a NoneType

In [3]:
cap = cv2.VideoCapture(0)
# intialize holistic model with only one hand
# (could be changed to more than one)
with mp_hands.Hands(max_num_hands=1) as hand:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor the image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = hand.process(image)

        # Landmark detection
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                    #[print(landmark) for landmark in handLms.landmark]
                    mp_drawing.draw_landmarks(image, handLms, mp_hands.HAND_CONNECTIONS)
                    

                        
        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

# Capture Landmarks and export to CSV

In [4]:
import csv
import os
import numpy as np

In [5]:
num_cords = len(results.multi_hand_landmarks[0].landmark)
num_cords

21

In [6]:
landmarks = ['class']
for val in range(1, num_cords+1):
    landmarks += ['x{}'.format(val), 'y{}'.format(val), 'z{}'.format(val)]

In [7]:
# reset csv
def reset_csv():
    f = open('hands-coords.csv', 'r+')
    f.truncate(0)

reset_csv()

In [8]:
with open('hands-coords.csv', mode='w', newline='') as f:
    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
    csv_writer.writerow(landmarks)

In [16]:
class_name = "You"
cap = cv2.VideoCapture(0)
# intialize hand model
with mp_hands.Hands(max_num_hands=1) as hand:

    while cap.isOpened():
        ret, frame = cap.read()

        # Recolor the image
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        image.flags.writeable = False
        # Make Detections
        results = hand.process(image)

        # Landmark detection
        image.flags.writeable = True
        image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        
        if results.multi_hand_landmarks:
            for handLms in results.multi_hand_landmarks:
                    mp_drawing.draw_landmarks(image, handLms, mp_hands.HAND_CONNECTIONS)

        if results.multi_hand_landmarks:
            # Export coordinates
            try:
                # Extracting hand landmarks
                # Extracting pose landmark
                detected_hand = results.multi_hand_landmarks[0].landmark
                hand_row = list(np.array([[landmark.x, landmark.y, landmark.z] for landmark in detected_hand]).flatten())
                #print(list([landmark for landmark in detected_hand]))

                # Concatanate rows
                row = hand_row
                row.insert(0, class_name)

                with open('../data/hands-coords.csv', mode='a', newline='') as f:
                    csv_writer = csv.writer(f, delimiter=',', quotechar='"', quoting=csv.QUOTE_MINIMAL)
                    csv_writer.writerow(row)

            except:
                pass

        cv2.imshow('Raw Webcam Feed', image)

        if cv2.waitKey(10) & 0xFF == ord('q'):
            break

cap.release()
cv2.destroyAllWindows()

In [15]:
# reset csv
# def reset_csv():
#     f = open('hands-coords.csv', 'r+')
#     f.truncate(0)

# reset_csv()